### Please refer to notebook Train_Validation_Test_Split for more details about problem statement.

This notebook expects that user has already run notebook Run_Model 4 times to save 4 models, each corresponding to one attribute.



## Importing Libraries

In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
try: 
  print('set memory growth')
  tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except: 
  # Invalid device or cannot modify virtual devices once initialized. 
  pass 

set memory growth


In [2]:
#from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle


import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import matplotlib.pyplot as plt
import cv2
import os


Using TensorFlow backend.


In [3]:
IMG_WIDTH=300
IMG_HEIGHT=300
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)

## Load the earlier save test data 

In [4]:
X_test_all = np.load('5000//X_test_all.npy')
y_test_all = np.load('5000//y_test_all.npy')


In [7]:
X_test_all= X_test_all.flatten()
X_test_all = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in X_test_all.tolist()]
X_test_all = np.array(X_test_all)
y_test_all = np.array(y_test_all)

print( X_test_all.shape)
print( y_test_all.shape)


(665, 300, 300, 3)
(665, 1)


## Load 4 Models. 1 Model for each attribute

In [8]:
num_attributes =4
model = [[] for i in range(num_attributes)]
model[0] = keras.models.load_model('./savemodels/glamorous_model.h5')
model[1] = keras.models.load_model('./savemodels/rugged_model.h5')
model[2] = keras.models.load_model('./savemodels/fun_model.h5')
model[3] = keras.models.load_model('./savemodels/healthy_model.h5')

## Testing

Since we are using one vs rest approach of multiclass classification, we need to create 4 classifiers one for each attribute. 

For test data, prediction from each classifier will be merged to give one label for each test sample. Suppose for a paricular test image, classifier 1 gives probability(0.1, prediction is that image does not belong to first attribute), classifier 2 gives probability (0.8, prediction is that image does belong to 2nd attribute), and so on. All the four predictions will be merge together to give a label correponding to a classifier with highest probability.

In [9]:
 
'''
 Take as input probability output of 4 classfiers coressponding to each class( We have 4 classes and 
 one classfier corresponding to each class )
 Predict the class label which have the highest probability 
 Return the class label corresponding to class which have highest probablity
'''
def give_ovr_class_label_output(y_pred):
    '''
    Stack 4 probabilities to get a numpy array 
    '''
    yclass= np.hstack((y_pred[0], y_pred[1],y_pred[2],y_pred[3]))
        
    y_class = np.array( yclass )
    '''
    Get the index corresponding to the max value of column
    '''
    idx = np.argmax(y_class, axis=-1)
    y_class = np.zeros(y_class.shape )
    '''
    Convert the one hot position into class label
    '''
    y_class[ np.arange(y_class.shape[0]), idx] = 1
    y_train_pred = np.argmax(y_class, axis=1)
    return y_train_pred


In [11]:
y_pred = [[] for i in range(num_attributes)]
for i in range(num_attributes):
    y_pred[i] = model[i].predict(X_test_all)
y_pred_label = give_ovr_class_label_output(y_pred)

## Print Confusion Matrix and Prediction Accuracy

In [16]:
print('Confusion Matrix')
cm = confusion_matrix(y_test_all, \
                      y_pred_label)
print(cm)

Confusion Matrix
[[155   2  51   0]
 [ 18 138  50   0]
 [ 38   3 166   0]
 [ 10   1  23  10]]


In [25]:
a = accuracy_score(y_pred_label,y_test_all)
print('Accuracy is:', a*100)

Accuracy is: 70.52631578947368


## Print Precison and Recall

In [17]:
def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()
    
def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()

def precision_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_precisions = 0
    for label in range(rows):
        sum_of_precisions += precision(label, confusion_matrix)
    return sum_of_precisions / rows

def recall_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_recalls = 0
    for label in range(columns):
        sum_of_recalls += recall(label, confusion_matrix)
    return sum_of_recalls / columns

In [20]:
print("label precision recall")
for label in range(4):
    print(f"{label:5d} {precision(label, cm):9.3f} {recall(label, cm):6.3f}")

label precision recall
    0     0.701  0.745
    1     0.958  0.670
    2     0.572  0.802
    3     1.000  0.227
